#### Face Detection & Face Recognition

mediapipe 패키지 설치

> pip install mediapipe

##### Webcam Recognition

In [2]:
# Webcam recognition

import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
      success, image = cap.read()
      if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        continue

      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = face_detection.process(image)

      # Draw the face detection annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      if results.detections:
        for detection in results.detections:
          mp_drawing.draw_detection(image, detection)

      # Flip the image horizontally for a selfie-view display.
      cv2.imshow('Face Detection', cv2.flip(image, 1)) # cv2.flip(image, 1) : image flip
      if cv2.waitKey(1) == ord('q'):
        break
      
cap.release()

##### Video Recognition

In [ ]:
# 라이브러리 불러오기
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('video/face_video.mp4')


with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
  # model_selection = 0 : 카메라로부터 2m 이내의 근거리에 적합, = 1 : 5m 이내의 근거리에 적합
  # min_detection_confidence : 신뢰도가 특정 %이상되면, 얼굴로 인식할지 정의 (threshold개념과 유사)

    while cap.isOpened():
      success, image = cap.read() # frame을 불러와서 success되면 image에 저장
      if not success:
        break

      # To improve performance, optionally mark the image as not writeable to
      # pass by reference.
      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = face_detection.process(image) # image로부터 얼굴을 검출해서 results로 반환

      # Draw the face detection annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

      if results.detections: # detection한 results가 있을 시 진행
        
        # 반복문 통해서 검출된 사람 얼굴만큼 draw 해줌
        # 6개 포인트(오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀) 검출
        for detection in results.detections:
          mp_drawing.draw_detection(image, detection)
          # print(detection) # detection 정보 확인

          # 특정 위치 가져오기(index 적용)
          keypoints = detection.location_data.relative_keypoints
          right_eye = keypoints[0]
          left_eye = keypoints[1]
          nose_tip = keypoints[2]

          h, w, _ = image.shape # height, width, channel : 이미지로부터 height, width 크기에 대한 정보 가져옴 (여기서 channel은 필요없으므로 _처리)

          # 이미지 내에서 실제 좌표(x, y) 설정
          # relative_keypoints에서 가져온 x, y좌표와 h, w을 곱하면 이미지에서 원하는 좌표를 얻을 수 있음
          right_eye = (int(right_eye.x * w), int(right_eye.y * h)) # x, y 좌표와 w, h의 곱을 정수형으로 변환 후 전체 튜플 형태로 변경
          left_eye = (int(left_eye.x * w), int(left_eye.y * h))
          nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))

          # 3개 포인트에 원으로 표시
          cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA)
          cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA)
          cv2.circle(image, nose_tip, 50, (0, 255, 255), 10, cv2.LINE_AA)


      # Flip the image horizontally for a selfie-view display & Video resize 0.5
      cv2.imshow('Face Detection', cv2.resize(image, None, fx = 0.5, fy = 0.5))

      if cv2.waitKey(1) == ord('q'):
        break
      
cap.release()
cv2.destroyAllWindows()

'''
<detection 정보>
- score : 신뢰도
- location_data : 1 frame에서 발견한 detection 위치
- relative_bounding_box : bounding box 위치
- relative_keypoints : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 좌표


label_id: 0
score: 0.972063422203064
location_data {
  format: RELATIVE_BOUNDING_BOX
  relative_bounding_box {
    xmin: 0.34010049700737
    ymin: 0.27738773822784424
    width: 0.16470429301261902
    height: 0.2928076982498169
  }
  relative_keypoints {
    x: 0.39748260378837585
    y: 0.3514038920402527
  }
  relative_keypoints {
    x: 0.472693532705307
    y: 0.3491246700286865
  }
  relative_keypoints {
    x: 0.4475392699241638
    y: 0.41251301765441895
  }
  relative_keypoints {
    x: 0.4445818364620209
    y: 0.4767593741416931
  }
  relative_keypoints {
    x: 0.3378288745880127
    y: 0.39561253786087036
  }
  relative_keypoints {
    x: 0.49736836552619934
    y: 0.38518768548965454
  }
}
'''